In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import HTML
from io import BytesIO
import base64
from pptx import Presentation
from pptx.util import Inches
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Pt
from datetime import datetime

# 맵 이미지를 Base64 문자열로 인코딩하는 함수
def plot_to_base64(x, y, values):
    colors = np.where(values < 0.3, 'red', np.where(values > 0.3, 'blue', 'gray'))
    fig, ax = plt.subplots(figsize=(2, 2), dpi=100)
    scatter = ax.scatter(x, y, s=100, c=colors, marker='s', edgecolor='black')
    ax.axis('off')
    
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    return base64.b64encode(buf.getvalue()).decode()

# HTML 테이블 생성 및 맵 이미지 삽입 함수
def create_html_table_with_maps(df):
    style = """
    <style>
        table {
            width: 40%; /* 조정된 테이블 너비 */
            border-collapse: collapse;
            margin: 10px 0;
            font-family: Arial, sans-serif;
        }
        th, td {
            padding: 4px;
            border: 1px solid #ccc;
            text-align: center; /* 모든 값 가운데 정렬 */
            font-size: 14px;
        }
        th {
            background-color: #f0f0f0;
        }
        img {
            width: 100%;
            height: auto;
        }
        .map-cell {
            width: 50px;
            height: 50px;
        }
        .low { background-color: red; } /* 0.5 이하 */
        .mid { background-color: grey; } /* 0.5~0.75 */
        .high { background-color: lightblue; } /* 0.75 이상 */
    </style>
    """
    html_str = style + '<table>'
    cat3_values = sorted(df['cat3'].unique())
    html_str += '<tr><td class="value">Value</td>' + ''.join([f'<td>{cat3}</td>' for cat3 in cat3_values]) + '</tr>'
    
    for i, val in enumerate([f'val{i}' for i in range(1, 6)], start=1):
        html_str += f'<tr><td class="value">{val}</td>'
        for cat3 in cat3_values:
            mean_val = df[df['cat3'] == cat3][val].mean()
            # 셀 배경색을 조건에 따라 결정
            cell_class = "low" if mean_val <= 0.5 else "mid" if mean_val <= 0.75 else "high"
            html_str += f'<td class="{cell_class}">{mean_val:.2f}</td>'
        html_str += '</tr>'
        html_str += f'<tr><td class="value">{val}_map</td>'
        for cat3 in cat3_values:
            subset = df[df['cat3'] == cat3]
            img_data = plot_to_base64(subset['x'], subset['y'], subset[val])
            html_str += f'<td class="map-cell"><img src="data:image/png;base64,{img_data}"></td>'
        html_str += '</tr>'
    html_str += '</table>'
    return html_str

def create_boxplot_base64(df, val_column):
    plt.figure(figsize=(10, 5))
    
    # catA 열의 고유값을 숫자형과 'Ref.'로 분리하여 정렬
    numeric_vals = sorted([x for x in df['catA'].unique() if str(x).isdigit()], key=int)
    ref_val = [x for x in df['catA'].unique() if x == 'Ref.']
    order = numeric_vals + ref_val  # 숫자형 값들을 먼저 정렬하고 'Ref.'를 마지막에 추가
    
    # 박스플롯 그리기: 정렬된 순서대로 표시
    sns.boxplot(x='catA', y=val_column, data=df, color='grey', order=order)

    # 각 박스 옆에 산점도 추가: 동일한 순서를 적용
    sns.stripplot(x='catA', y=val_column, data=df, color='blue', alpha=0.5, jitter=True, dodge=True, order=order)

    plt.title(f'{val_column} boxplot by catA')
    buf = BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    plt.close()
    return base64.b64encode(buf.getvalue()).decode()

# stats table 생성하고 html로 만드는 함수
def calculate_and_format_stats_html(df, val_columns):
    html_str = "<style>table {border-collapse: collapse; width: 40%;} th, td {border: 1px solid #ddd; padding: 8px; text-align: center;} th {background-color: #f2f2f2;}</style>"
    
    # cat1이 'ProductA'인 행만 필터링
    df = df[df['cat1'] == 'ProductA']

    for val_column in val_columns:
        # 통계치를 수동으로 계산
        stats = {
            'count': df.groupby('cat3')[val_column].count().astype(int).apply(lambda x: f"{x} point"),
            'median': df.groupby('cat3')[val_column].median().round(2),
            'std': df.groupby('cat3')[val_column].std().round(2),
            'max': df.groupby('cat3')[val_column].max().round(2),
            'p90': df.groupby('cat3')[val_column].apply(lambda x: np.percentile(x, 90)).round(2),
            'p10': df.groupby('cat3')[val_column].apply(lambda x: np.percentile(x, 10)).round(2),
            'min': df.groupby('cat3')[val_column].min().round(2),
            '<0.5 count': df[df[val_column] < 0.5].groupby('cat3')[val_column].count().astype(int).apply(lambda x: f"{x} point")
        }
        # DataFrame으로 변환
        stats_df = pd.DataFrame(stats).reset_index()
        # 'cat3' 열 제거
        #stats_df = stats_df.drop(columns=['cat3'])
        # HTML로 변환
        stats_html = stats_df.T.to_html(header=False, index=True)
        html_str += f"<h3>{val_column} Statistics Boxplot and stats table</h3>" + stats_html
    return html_str

# boxplot 및 stats table 합성 HTML 컨텐츠 생성 및 파일 저장
def create_full_html_content(df):
    val_columns = [f'val{i}' for i in range(1, 6)]
    full_html_content = ""
    for val_column in val_columns:
        boxplot_img_data = create_boxplot_base64(df, val_column)
        full_html_content += f'<img src="data:image/png;base64,{boxplot_img_data}" style="width:38%; display: block; margin-left: 35; margin-right: auto;">'
        stats_html = calculate_and_format_stats_html(df, [val_column])
        full_html_content += stats_html
    return full_html_content

# 데이터 프레임 생성
np.random.seed(0)
data = {
    'cat1': np.random.choice(['ProductA', 'ProductB', 'ProductC', 'ProductD', 'ProductE', 'ProductF', 'ProductG'], 3000),
    'cat2': np.random.choice(['Setting1', 'Setting2', 'Setting3', 'Setting4', 'Setting5', 'Setting6', 'Setting7', 'Setting8'], 3000),
    'cat3': np.random.randint(1, 11, 3000),
    'x': np.random.randint(1, 11, 3000),
    'y': np.random.randint(1, 11, 3000),
    'datetime': pd.date_range(start='2021-01-01', periods=3000, freq='D')
}
df = pd.DataFrame(data)
# 사인(sin)과 코사인(cos) 기반 데이터 및 이상치 생성
t = np.linspace(0, 2 * np.pi, 3000)  # 0부터 2π까지의 범위에서 3000개의 점 생성
new_vals = {}
for val_num in range(1, 40):
    if val_num % 2 == 0:  # 짝수 번호의 경우 사인 함수 사용
        values = np.sin(t * val_num) / 2 + 0.5  # 0과 1 사이의 값으로 조정
    else:  # 홀수 번호의 경우 코사인 함수 사용
        values = np.cos(t * val_num) / 2 + 0.5  # 0과 1 사이의 값으로 조정
    # 이상치 추가
    outlier_indices = np.random.choice(range(3000), size=10, replace=False)  # 이상치를 추가할 100개의 랜덤 인덱스
    values[outlier_indices] = values[outlier_indices] + np.random.choice([-5, 5], size=10)  # 이상치 추가
    new_vals[f'val{val_num}'] = values

# 새로운 vals 칼럼들을 데이터 프레임에 추가
df = pd.concat([df, pd.DataFrame(new_vals)], axis=1)

# 새로운 열 catA 추가: cat1이 'ProductA'이면 cat3의 값을, 아니면 'Ref.'를 할당
df['catA'] = np.where(df['cat1'] == 'ProductA', df['cat3'], 'Ref.')

# cat1이 'ProductA'인 행만 필터링
filtered_df = df[df['cat1'] == 'ProductA']

# 1. 데이터프레임에서 테이블과 맵 이미지 생성
html_table_with_maps = create_html_table_with_maps(filtered_df)

# 2. 데이터프레임에서 BOXPLOT과 Stat 테이블 생성
html_boxplot_with_stats = create_full_html_content(df)

full_html_content = "<style>table {width: 100%; border-collapse: collapse;} th, td {border: 1px solid #ddd; padding: 8px; text-align: center;} th {background-color: #f2f2f2;}</style>"
full_html_content += "<p>아래는 score 및 map table.</p>"
full_html_content += html_table_with_maps
full_html_content += "<p>아래는 val boxplot 및 stats table.</p>"
full_html_content += html_boxplot_with_stats

HTML(full_html_content)

# HTML 파일 저장
html_file_path = './sum_test.html'
with open(html_file_path, 'w') as file:
    file.write(full_html_content)

print(f"HTML 파일이 저장된 위치: {html_file_path}")

# 'cat1', 'cat2', 'cat3', 'x', 'y', 'datetime' 열을 제외한 리스트 구하기
excluded_columns = ['cat1', 'cat2', 'cat3', 'catA', 'x', 'y', 'datetime']
timeseries_columns = [col for col in df.columns if col not in excluded_columns]

# PPTX 프레젠테이션 생성
prs = Presentation()

# 요약 슬라이드 추가 함수 (수정됨)
def add_summary_slide(prs, cat1_value, cat3_values, link, image_path):
    slide_layout = prs.slide_layouts[6]  # 선택한 레이아웃
    slide = prs.slides.add_slide(slide_layout)  # 맨 뒤에 슬라이드 추가
    
    # 제목 텍스트 박스 생성 및 설정
    title_box = slide.shapes.add_textbox(Inches(1), Inches(1), Inches(8), Inches(1))
    title_tf = title_box.text_frame
    title_p = title_tf.paragraphs[0]
    title_p.text = "Summary Information"
    title_p.font.size = Pt(42)  # 제목 글씨 크기 설정
    title_p.font.bold = True

    # 세부 정보 텍스트 박스 생성 및 설정
    details_box = slide.shapes.add_textbox(Inches(1), Inches(2), Inches(8), Inches(4))
    details_tf = details_box.text_frame
    details_p = details_tf.add_paragraph()
    
    # cat3 값들 앞에 '#' 붙여서 문자열로 변환
    cat3_values_str = ', '.join([f"#{val}" for val in sorted(cat3_values)])
    
    details_p.text = f"Selected cat1 value: {cat1_value}\nUnique cat3 values: {cat3_values_str}\nGenerated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\nVisit: {link}"
    details_p.font.size = Pt(20)  # 세부 정보 글씨 크기 설정

    # 이미지 추가
    # PPTX의 우측 하단에 이미지를 배치하기 위한 좌표와 크기 설정
    # 예시: 이미지를 슬라이드의 우측 하단에 너비 2인치, 높이 2인치 크기로 배치
    slide_width = prs.slide_width
    slide_height = prs.slide_height
    image_width = Inches(2.5)
    image_height = Inches(2.5)
    image_left = slide_width - image_width - Inches(0.5)  # 오른쪽 여백을 0.5인치로 설정
    image_top = slide_height - image_height - Inches(0.5)  # 하단 여백을 0.5인치로 설정

    slide.shapes.add_picture(image_path, image_left, image_top, image_width, image_height)

# 요약 슬라이드를 맨 처음에 추가
selected_cat1_value = "ProductA"
unique_cat3_values = df['cat3'].unique()
image_path = "./images.jpg"  # 이미지 파일 경로
add_summary_slide(prs, selected_cat1_value, unique_cat3_values, "http://abc.com", image_path)

# 각 25개 열에 대해 이미지 생성 및 PPTX에 삽입
for i in range(0, len(timeseries_columns), 25):
    current_columns = timeseries_columns[i:i+25]
    fig, axs = plt.subplots(5, 5, figsize=(15, 15), dpi=100)  # 5x5 그리드 생성
    
    # 현재 그룹의 각 열에 대해 플롯 생성
    for j, column in enumerate(current_columns):
        ax = axs.flatten()[j]
        ax.scatter(df.index, df[column], alpha=0.5)
        ax.set_title(column, fontsize=12)

    # 나머지 빈칸은 비워둠
    for k in range(j + 1, 25):
        axs.flatten()[k].axis('off')

    plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)

    # 슬라이드 추가 및 이미지 삽입
    slide_layout = prs.slide_layouts[6]  # 가장 기본적인 레이아웃
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.add_picture(buf, Inches(0), Inches(0), width=prs.slide_width, height=prs.slide_height)

# PPTX 파일 저장
pptx_file_path = "timeseries_plots_combined.pptx"
prs.save(pptx_file_path)
print(f"PPTX 파일이 저장된 위치: {pptx_file_path}")

HTML 파일이 저장된 위치: ./sum_test.html
PPTX 파일이 저장된 위치: timeseries_plots_combined.pptx


HTML 파일이 저장된 위치: ./sum_test.html
